# Install

In [ ]:
!pip install sentencepiece

# Evn

In [ ]:
import os
import random
import shutil
import json
import zipfile
import math
import copy
import collections
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sentencepiece as spm
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm.notebook import tqdm

In [ ]:
# random seed initialize
random_seed = 1234
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

In [ ]:
!nvidia-smi

In [ ]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# data dir
data_dir = '/content/drive/MyDrive/Data/nlp' # change '/content/drive/MyDrive/Data/nlp' to your drive path 
os.listdir(data_dir)

In [ ]:
# korean wiki dir
kowiki_dir = os.path.join(data_dir, 'kowiki')
if not os.path.exists(kowiki_dir):
    os.makedirs(kowiki_dir)
os.listdir(kowiki_dir)

# Vocabualry & Config

In [ ]:
# vocab loading
vocab = spm.SentencePieceProcessor()
vocab.load(os.path.join(data_dir, 'ko_32000.model'))

In [ ]:
n_vocab = len(vocab)  # number of vocabulary
n_seq = 256  # number of sequence
d_model = 256  # dimension of model

# 모델링

In [ ]:
# 입력 문장
sentences = [
    '나는 오늘 기분이 좋아 나는 오늘 우울해',
    '나는 오늘 행복해 나는 오늘 즐거워'
]

In [ ]:
# 각 문장을 띄어쓰기 단위로 분할
words = []
for sentence in sentences:
    words.extend(sentence.split())

# 중복 단어 제거
words = list(dict.fromkeys(words))

# 각 단어별 고유한 번호 부여
word_to_id = {'[PAD]': 0, '[UNK]': 1, '[BOS]': 2, '[EOS]': 3}
for word in words:
    word_to_id[word] = len(word_to_id)

# 각 숫자별 단어 부여
id_to_word = {_id:word for word, _id in word_to_id.items()}

word_to_id, id_to_word

In [ ]:
# 학습용 입력 데이터 생성
# [[4, 5, 6, 7, 4, 5, 8], [4, 5, 9, 4, 5, 10]]
train_ids = []
for sentence in sentences:
    train_ids.append([word_to_id[word] for word in sentence.split()])

# id를 입력과 정답으로 저장
# ([[2, 4, 5, 6, 7, 4, 5, 8], [2, 4, 5, 9, 4, 5, 10]],
#  [[4, 5, 6, 7, 4, 5, 8, 3], [4, 5, 9, 4, 5, 10, 3]])
train_inputs, train_labels = [], []
for train_id in train_ids:
    train_inputs.append([word_to_id['[BOS]']] + train_id)
    train_labels.append(train_id + [word_to_id['[EOS]']])

# 문장의 길이를 모두 동일하게 변경 (최대길이 8)
for row in train_inputs:
    row += [0] * (8 - len(row))

# 문장의 길이를 모두 동일하게 변경 (최대길이 8)
for row in train_labels:
    row += [0] * (8 - len(row))

# train inputs을 numpy array로 변환
train_inputs = np.array(train_inputs)
train_labels = np.array(train_labels)

train_inputs, train_labels

In [ ]:
# 단어를 벡터로 변환
# AttributeError: 'list' object has no attribute 'dtype' -> 위에서 train_inputs을 numpy의 array 형식으로 만들어주지 않으면 생기는 에러
embedding = tf.keras.layers.Embedding(len(word_to_id), 4)
hidden = embedding(train_inputs)  # (bs, n_seq, 4)
hidden

In [ ]:
# LSTM 실행
lstm = tf.keras.layers.LSTM(units=5, return_sequences=True)
hidden = lstm(hidden)
hidden

In [ ]:
len(word_to_id)

In [ ]:
# 다음단어 예측
dense = tf.keras.layers.Dense(len(word_to_id), activation=tf.nn.softmax)
outputs = dense(hidden)
outputs

In [ ]:
def build_model(n_vocab, d_model, n_seq):
    """
    문장 유사도 비교 모델
    :param n_vocab: vocabulary 단어 수
    :param d_model: 단어를 의미하는 벡터의 차원 수
    :param n_seq: 문장 길이 (단어 수)
    """
    inputs = tf.keras.layers.Input((n_seq,))  # (bs, n_seq)
    #######################################################
    # 입력 단어를 vector로 변환
    embedding = tf.keras.layers.Embedding(n_vocab, d_model)
    hidden = embedding(inputs)  # (bs, n_seq, d_model)
    # LSTM
    lstm = tf.keras.layers.LSTM(units=d_model * 2, return_sequences=True)
    hidden = lstm(hidden)  # (bs, n_seq, d_model * 2)
    # 다음단어 확률 분포
    dense = tf.keras.layers.Dense(n_vocab, activation=tf.nn.softmax)
    outputs = dense(hidden)
    #######################################################
    # 학습할 모델 선언
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
# 모델 생성
model = build_model(len(word_to_id), 4, 8)
# 모델 내용 그래프 출력
tf.keras.utils.plot_model(model, 'model.png', show_shapes=True)

# Processing

In [ ]:
# 파일 내용 확인
with zipfile.ZipFile(os.path.join(kowiki_dir, 'kowiki.txt.zip')) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(f):
            line = line.decode('utf-8').strip()
            print(line)
            if i >= 100:
                break

In [ ]:
# 파일 내용 확인 (주제단위)
with zipfile.ZipFile(os.path.join(kowiki_dir, 'kowiki.txt.zip')) as z:
    with z.open('kowiki.txt') as f:
        doc = []
        for i, line in enumerate(f):
            line = line.decode('utf-8').strip()
            if len(line) == 0:
                if len(doc) > 0:
                    break
            else:
                doc.append(line)
doc

In [ ]:
def create_train_instance(vocab, n_seq, doc):
    """
    create train instance
    :param vocab: vocabulary object
    :param n_seq: sequece number
    :param doc: wiki document
    :return: train instance list
    """
    n_max = n_seq - 1  # [BOS] or [EOS]
    instance_list = []

    chunk = []
    chunk_len = 0
    for i, line in enumerate(doc):
        ##################################
        # print(line)
        tokens = vocab.encode_as_pieces(line)
        chunk.append(tokens)
        chunk_len += len(tokens)
        if n_max <= chunk_len or i >= len(doc) -1:
            # print()
            # print(chunk_len, chunk)
            # 문장의 배열을 하나의 배열로 변경
            instance = []
            for tokens in chunk:
                instance.extend(tokens)
            # 길이가 긴 경우 길이를 자름
            instance = instance[:n_max]
            # print(len(instance), instance)
            # instance 저장
            instance_list.append(instance)
            # chunk 초기화
            chunk = []
            chunk_len = 0
        ##################################
    return instance_list

In [ ]:
# instance 동작 확인
instance_list = create_train_instance(vocab, n_seq, doc)
for instance in instance_list:
    print(len(instance), instance)

In [ ]:
# instance를 json 형태로 저장하는 함수
def save_instance(vocab, n_seq, doc, o_f):
    instance_list = create_train_instance(vocab, n_seq, doc)
    for instance in instance_list:
        o_f.write(json.dumps({'token': instance}, ensure_ascii=False))
        o_f.write('\n')

In [ ]:
# 전체 문서에 대한 instance 생성
with open(os.path.join(kowiki_dir, 'kowiki_lm.json'), 'w') as o_f:
    with zipfile.ZipFile(os.path.join(kowiki_dir, 'kowiki.txt.zip')) as z:
        with z.open('kowiki.txt') as f:
            doc = []
            for i, line in enumerate(tqdm(f)):
                line = line.decode('utf-8').strip()
                if len(line) == 0:
                    if len(doc) > 0:
                        save_instance(vocab, n_seq, doc, o_f)
                        doc = []
                        # break
                else:
                    doc.append(line)
            if len(doc) > 0:
                save_instance(vocab, n_seq, doc, o_f)

In [ ]:
# 파일 라인수 확인
n_line = 0
with open(os.path.join(kowiki_dir, 'kowiki_lm.json')) as f:
    for line in f:
        n_line += 1
        if n_line <= 10:
            print(line)
n_line

# Sample Data Project

# 1. Data

In [ ]:
def load_data(vocab, n_seq):
    """
    Language Model 학습 데이터 생성
    :param vocab: vocabulary object
    :param n_seq: number of sequence
    :return inputs_1: input data 1
    :return inputs_2: input data 2
    :return labels: label data
    """
    # line 수 조회
    # n_line = 0
    # with open(os.path.join(kowiki_dir, 'kowiki_lm.json')) as f:
    #     for line in f:
    #         n_line += 1
    # 10개로 제한
    n_data = 10
    # 빈 데이터 생성
    inputs = np.zeros((n_data, n_seq)).astype(np.int32)
    labels = np.zeros((n_data, n_seq)).astype(np.int32)

    with open(os.path.join(kowiki_dir, 'kowiki_lm.json')) as f:
        for i, line in enumerate(f):
            if i >= n_data:
                break
            ##################################
            data = json.loads(line)
            print()
            print(data)
            token_id = [vocab.piece_to_id(p) for p in data['token']]
            print(len(token_id), token_id)
            # input id
            input_id = [vocab.bos_id()] + token_id
            input_id += [0] * (n_seq - len(input_id))
            print(len(input_id), input_id)
            # label id
            label_id = token_id + [vocab.eos_id()]
            label_id += [0] * (n_seq - len(label_id))
            print(len(label_id), label_id)
            # 값 저장
            inputs[i] = input_id
            labels[i] = label_id
            ##################################

    return inputs, labels

In [ ]:
# train data 생성
train_inputs, train_labels = load_data(vocab, n_seq)
train_inputs, train_labels

# 2. Loss & Acc

In [ ]:
# 4개 의 단어를 예측하기 위한 정답과 예측값 가정
# 정답
y_true = np.random.randint(1, 3, (1, 20)).astype(np.float32)
y_true[:, 8:] = 0
# 예측 값
y_pred = np.random.random((1, 20, 4)).astype(np.float32)
y_pred = tf.nn.softmax(y_pred, axis=-1).numpy()  # 확률 값으로 변경

y_true, y_pred

In [ ]:
def lm_loss(y_true, y_pred):
    """
    pad 부분을 제외하고 loss를 계산하는 함수
    :param y_true: 정답
    :param y_pred: 예측 값
    :retrun loss: pad 부분이 제외된 loss 값
    """
    # sparse_entropy = tf.keras.losses.SparseCategoricalCrossentropy()
    sparse_entropy = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    loss = sparse_entropy(y_true, y_pred)
    # mask 계산
    # print(y_true)
    mask = tf.not_equal(y_true, 0)
    mask = tf.cast(mask, tf.float32)
    # print(mask)
    loss *= mask
    return loss

In [ ]:
# lm_loss 함수 확인
loss = lm_loss(y_true, y_pred)
print(loss, np.mean(loss))

In [ ]:
def lm_acc(y_true, y_pred):
    """
    pad 부분을 제외하고 accuracy를 계산하는 함수
    :param y_true: 정답
    :param y_pred: 예측 값
    :retrun loss: pad 부분이 제외된 accuracy 값
    """
    y_true = tf.cast(y_true, tf.float32)
    # print(y_true)
    y_pred_class = tf.cast(tf.argmax(y_pred, axis=-1), tf.float32)
    # print(y_pred_class)
    matches = tf.cast(tf.equal(y_true, y_pred_class), tf.float32)
    # print(matches)
    # mask 계산
    mask = tf.not_equal(y_true, 0)
    mask = tf.cast(mask, tf.float32)
    # print(mask)
    # match에 mask 적용
    matches *= mask
    # print(matches)
    # accuracy = tf.reduce_sum(matches) / tf.maximum(tf.reduce_sum(tf.ones_like(matches)), 1)
    # 분모는 mask의 1인 개수
    accuracy = tf.reduce_sum(matches) / tf.maximum(tf.reduce_sum(mask), 1)
    return accuracy

In [ ]:
# lm_acc 함수 확인
accuracy = lm_acc(y_true, y_pred)
print(accuracy)

# 학습

In [ ]:
# 모델 생성
model = build_model(len(vocab), d_model, n_seq)
# 모델 내용 그래프 출력
tf.keras.utils.plot_model(model, 'model.png', show_shapes=True)

In [ ]:
# 모델 loss, optimizer, metric 정의
model.compile(loss=lm_loss, optimizer='adam', metrics=[lm_acc])

In [ ]:
# early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='lm_acc', patience=50)
# save weights callback
save_weights = tf.keras.callbacks.ModelCheckpoint(os.path.join(kowiki_dir, 'lm.hdf5'),
                                                  monitor='lm_acc',
                                                  verbose=1,
                                                  save_best_only=True,
                                                  mode="max",
                                                  save_freq="epoch",
                                                  save_weights_only=True)
# csv logger
csv_logger = tf.keras.callbacks.CSVLogger(os.path.join(kowiki_dir, 'lm.csv'))

In [ ]:
# 모델 학습
history = model.fit(train_inputs,
                    train_labels,
                    epochs=500,
                    batch_size=128,
                    callbacks=[early_stopping, save_weights, csv_logger])

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['lm_acc'], 'g-', label='acc')
plt.xlabel('Epoch')
plt.legend()

plt.show()

# Inference

In [ ]:
# 모델 생성
model = build_model(len(vocab), d_model, n_seq)
# train weight로 초기화
model.load_weights(os.path.join(kowiki_dir, 'lm.hdf5'))

In [ ]:
def do_next(vocab, model, n_seq, string):
    """
    다음단어 예측
    :param vocab: vocab
    :param model: model
    :param n_seq: number of seqence
    :param string: inpust string
    """
    n_max = n_seq - 1
    
    tokens = vocab.encode_as_pieces(string)
    start_idx = len(tokens)
    token_id = [vocab.piece_to_id(p) for p in tokens][:n_max]
    token_id = [vocab.bos_id()] + token_id
    token_id += [0] * (n_seq - len(token_id))
    assert len(token_id) == n_seq
    print(tokens)
    print(start_idx, ':', token_id)

    ###################################
    result = model.predict(np.array([token_id]))
    print(result.shape)
    prob = result[0][start_idx]
    print(prob)

    # 정렬 (작 --> 큰)
    max_args = np.argsort(prob)[-10:]
    print(max_args)
    max_args = list(max_args)
    # 정렬 (큰 --> 작)
    max_args.reverse()
    print(max_args)

    next_prob = []
    for i in max_args:
        w = vocab.id_to_piece(int(i))
        p = prob[i]
        next_prob.append((w, p))
    ###################################
    return next_prob

In [ ]:
string = '대한민국'
do_next(vocab, model, n_seq, string)

In [ ]:
while True:
    string = input('시작 문장 > ')
    string = string.strip()
    if len(string) == 0:
        break
    next_prob = do_next(vocab, model, n_seq, string)
    for w, p in next_prob:
        print(f'{w}: {p}')
    print()

In [ ]:
def do_generate(vocab, model, n_seq, string):
    """
    문장생성
    :param vocab: vocab
    :param model: model
    :param n_seq: number of seqence
    :param string: inpust string
    """
    n_max = n_seq - 1
    tokens = vocab.encode_as_pieces(string)
    start_idx = len(tokens)
    token_id = [vocab.piece_to_id(p) for p in tokens][:n_max]
    token_id = [vocab.bos_id()] + token_id
    token_id += [0] * (n_seq - len(token_id))
    assert len(token_id) == n_seq

    for _ in range(start_idx, n_seq - 1):
        print(token_id)
        #########################################
        outputs = model.predict(np.array([token_id]))
        prob = outputs[0][start_idx]
        # word_id = int(np.argmax(prob))
        word_id = int(np.random.choice(len(vocab), 1, p=prob)[0])
        if word_id == vocab.eos_id():
            break
        token_id[start_idx + 1] = word_id
        start_idx += 1
        #########################################
    predict_id = token_id[1:start_idx + 1]
    predict_str = vocab.decode_ids(predict_id)
    print(predict_id, predict_str)
    return predict_str

In [ ]:
string = '대한민국'
do_generate(vocab, model, n_seq, string)

In [ ]:
while True:
    string = input('시작 문장 > ')
    string = string.strip()
    if len(string) == 0:
        break
    predict_str = do_generate(vocab, model, n_seq, string)
    print(predict_str)

# All Data Project

# 1. Data

In [ ]:
def load_data(vocab, n_seq):
    """
    Language Model 학습 데이터 생성
    :param vocab: vocabulary object
    :param n_seq: number of sequence
    :return inputs_1: input data 1
    :return inputs_2: input data 2
    :return labels: label data
    """
    # line 수 조회
    n_line = 0
    with open(os.path.join(kowiki_dir, 'kowiki_lm.json')) as f:
        for line in f:
            n_line += 1
    # 최대 100,000개 데이터
    n_data = min(n_line, 100000)
    # 빈 데이터 생성
    inputs = np.zeros((n_data, n_seq)).astype(np.int32)
    labels = np.zeros((n_data, n_seq)).astype(np.int32)

    with open(os.path.join(kowiki_dir, 'kowiki_lm.json')) as f:
        for i, line in enumerate(tqdm(f, total=n_data)):
            if i >= n_data:
                break
            data = json.loads(line)
            token_id = [vocab.piece_to_id(p) for p in data['token']]
            # input id
            input_id = [vocab.bos_id()] + token_id
            input_id += [0] * (n_seq - len(input_id))
            # label id
            label_id = token_id + [vocab.eos_id()]
            label_id += [0] * (n_seq - len(label_id))
            # 값 저장
            inputs[i] = input_id
            labels[i] = label_id

    return inputs, labels

In [ ]:
# train data 생성
train_inputs, train_labels = load_data(vocab, n_seq)
train_inputs, train_labels

# 2. Loss & Acc

In [ ]:
def lm_loss(y_true, y_pred):
    """
    pad 부분을 제외하고 loss를 계산하는 함수
    :param y_true: 정답
    :param y_pred: 예측 값
    :retrun loss: pad 부분이 제외된 loss 값
    """
    loss = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)(y_true, y_pred)
    mask = tf.not_equal(y_true, 0)
    mask = tf.cast(mask, tf.float32)
    loss *= mask
    return loss

In [ ]:
def lm_acc(y_true, y_pred):
    """
    pad 부분을 제외하고 accuracy를 계산하는 함수
    :param y_true: 정답
    :param y_pred: 예측 값
    :retrun loss: pad 부분이 제외된 accuracy 값
    """
    y_true = tf.cast(y_true, tf.float32)
    y_pred_class = tf.cast(tf.argmax(y_pred, axis=-1), tf.float32)
    matches = tf.cast(tf.equal(y_true, y_pred_class), tf.float32)
    mask = tf.not_equal(y_true, 0)
    mask = tf.cast(mask, tf.float32)
    matches *= mask
    accuracy = tf.reduce_sum(matches) / tf.maximum(tf.reduce_sum(mask), 1)
    return accuracy

# 3. 학습

In [ ]:
# 모델 loss, optimizer, metric 정의
model.compile(loss=lm_loss, optimizer='adam', metrics=[lm_acc])

In [ ]:
# early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='lm_acc', patience=50)
# save weights callback
save_weights = tf.keras.callbacks.ModelCheckpoint(os.path.join(kowiki_dir, 'lm.hdf5'),
                                                  monitor='lm_acc',
                                                  verbose=1,
                                                  save_best_only=True,
                                                  mode="max",
                                                  save_freq="epoch",
                                                  save_weights_only=True)
# csv logger
csv_logger = tf.keras.callbacks.CSVLogger(os.path.join(kowiki_dir, 'lm.csv'))

In [ ]:
# 모델 학습
history = model.fit(train_inputs,
                    train_labels,
                    epochs=2,
                    batch_size=64,
                    callbacks=[early_stopping, save_weights, csv_logger])

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['lm_acc'], 'g-', label='acc')
plt.xlabel('Epoch')
plt.legend()

plt.show()

# Inference

In [ ]:
# 모델 생성
model = build_model(len(vocab), d_model, n_seq)
# train weight로 초기화
model.load_weights(os.path.join(kowiki_dir, 'lm.hdf5'))

In [ ]:
def do_next(vocab, model, n_seq, string):
    """
    다음단어 예측
    :param vocab: vocab
    :param model: model
    :param n_seq: number of seqence
    :param string: inpust string
    """
    n_max = n_seq - 1
    
    tokens = vocab.encode_as_pieces(string)
    start_idx = len(tokens)
    token_id = [vocab.piece_to_id(p) for p in tokens][:n_max]
    token_id = [vocab.bos_id()] + token_id
    token_id += [0] * (n_seq - len(token_id))
    assert len(token_id) == n_seq

    result = model.predict(np.array([token_id]))
    prob = result[0][start_idx]
    max_args = np.argsort(prob)[-10:]
    max_args = list(max_args)
    max_args.reverse()

    next_prob = []
    for i in max_args:
        w = vocab.id_to_piece(int(i))
        p = prob[i]
        next_prob.append((w, p))
    return next_prob

In [ ]:
while True:
    string = input('시작 문장 > ')
    string = string.strip()
    if len(string) == 0:
        break
    next_prob = do_next(vocab, model, n_seq, string)
    for w, p in next_prob:
        print(f'{w}: {p}')
    print()

In [ ]:
def do_generate(vocab, model, n_seq, string):
    """
    문장생성
    :param vocab: vocab
    :param model: model
    :param n_seq: number of seqence
    :param string: inpust string
    """
    n_max = n_seq - 1
    tokens = vocab.encode_as_pieces(string)
    start_idx = len(tokens)
    token_id = [vocab.piece_to_id(p) for p in tokens][:n_max]
    token_id = [vocab.bos_id()] + token_id
    token_id += [0] * (n_seq - len(token_id))
    assert len(token_id) == n_seq

    for _ in range(start_idx, n_seq - 1):
        outputs = model.predict(np.array([token_id]))
        prob = outputs[0][start_idx]
        word_id = int(np.random.choice(len(vocab), 1, p=prob)[0])
        # word_id = np.argmax(prob)
        if word_id == vocab.eos_id():
            break
        token_id[start_idx + 1] = word_id
        start_idx += 1
    predict_id = token_id[1:start_idx + 1]
    predict_str = vocab.decode_ids(predict_id)
    return predict_str

In [ ]:
while True:
    string = input('시작 문장 > ')
    string = string.strip()
    if len(string) == 0:
        break
    predict_str = do_generate(vocab, model, n_seq, string)
    print(predict_str)